In [16]:
import pandas as pd
import numpy as np
from scipy.stats import shapiro, ttest_ind, mannwhitneyu
from scipy.spatial.distance import cosine
import ast

# === Загрузка данных ===
df = pd.read_csv('final_pipline.csv')

# === Параметры ===
groups = ['control', 'patient']
exclude_cols = ['filepath', 'group']
vector_features = ['mfcc', 'lpc']

# === Функции для проверки данных ===
def can_perform_test(data):
    """Проверка, можно ли выполнять статистический тест:
    - минимум 3 значения
    - более 1 уникального значения (не нулевой разброс)
    """
    return len(data) >= 3 and len(np.unique(data)) > 1

def safe_parse_vector(x):
    try:
        vec = ast.literal_eval(x)
        if isinstance(vec, list):
            vec = [float(i) for i in vec if np.isfinite(float(i))]
            return np.array(vec) if len(vec) > 0 else np.nan
    except (ValueError, SyntaxError, TypeError):
        return np.nan
    return np.nan

def parse_vector_column(series):
    return series.dropna().apply(lambda x: safe_parse_vector(x) if isinstance(x, str) and x.startswith('[') else np.nan)

# === Обработка обычных (скалярных) признаков ===
scalar_variables = [col for col in df.columns if col not in exclude_cols + vector_features]
results = []

for var in scalar_variables:
    data_control = pd.to_numeric(df[df['group'] == groups[0]][var], errors='coerce').replace([np.inf, -np.inf], np.nan).dropna()
    data_patient = pd.to_numeric(df[df['group'] == groups[1]][var], errors='coerce').replace([np.inf, -np.inf], np.nan).dropna()

    if not can_perform_test(data_control) or not can_perform_test(data_patient):
        results.append({
            'Переменная': var,
            'Тип анализа': 'Скалярный',
            'Контрольная группа (M±SD)': 'Нет вариации или недостаточно данных',
            'Группа пациентов (M±SD)': 'Нет вариации или недостаточно данных',
            'Тест на нормальность (p)': '-',
            'Стат. тест и p-value': '-',
            'Значимость': 'Нет'
        })
        continue

    # --- Тест на нормальность ---
    try:
        p_norm_control = shapiro(data_control)[1]
        p_norm_patient = shapiro(data_patient)[1]
    except Exception:
        p_norm_control = p_norm_patient = np.nan

    normal_control = (p_norm_control > 0.05) if not np.isnan(p_norm_control) else False
    normal_patient = (p_norm_patient > 0.05) if not np.isnan(p_norm_patient) else False

    # --- Выбор теста ---
    if normal_control and normal_patient:
        stat_test = 't-test'
        stat, p_value = ttest_ind(data_control, data_patient, nan_policy='omit')
    else:
        stat_test = 'Mann–Whitney'
        stat, p_value = mannwhitneyu(data_control, data_patient, alternative='two-sided')

    mean_std_control = f"{data_control.mean():.2f} ± {data_control.std():.2f}"
    mean_std_patient = f"{data_patient.mean():.2f} ± {data_patient.std():.2f}"
    significance = 'Да' if p_value < 0.05 else 'Нет'

    results.append({
        'Переменная': var,
        'Тип анализа': 'Скалярный',
        'Контрольная группа (M±SD)': mean_std_control,
        'Группа пациентов (M±SD)': mean_std_patient,
        'Тест на нормальность (p)': f"{p_norm_control:.3f} / {p_norm_patient:.3f}" if not np.isnan(p_norm_control) and not np.isnan(p_norm_patient) else '-',
        'Стат. тест и p-value': f"{stat_test}, p={p_value:.3f}",
        'Значимость': significance
    })

# === Обработка векторных признаков ===
def analyze_cosine_distance(column_name):
    control_vectors = parse_vector_column(df[df['group'] == 'control'][column_name]).dropna()
    patient_vectors = parse_vector_column(df[df['group'] == 'patient'][column_name]).dropna()

    if len(control_vectors) == 0 or len(patient_vectors) == 0:
        return {
            'Переменная': column_name,
            'Тип анализа': 'Векторный',
            'Контрольная группа (M±SD)': 'Недостаточно данных',
            'Группа пациентов (M±SD)': 'Недостаточно данных',
            'Тест на нормальность (p)': '-',
            'Стат. тест и p-value': 'Недостаточно данных',
            'Значимость': 'Нет'
        }

    control_mean_vector = np.mean(list(control_vectors), axis=0)

    control_distances = control_vectors.apply(lambda x: cosine(x, control_mean_vector))
    patient_distances = patient_vectors.apply(lambda x: cosine(x, control_mean_vector))

    control_distances = control_distances.replace([np.inf, -np.inf], np.nan).dropna()
    patient_distances = patient_distances.replace([np.inf, -np.inf], np.nan).dropna()

    # Проверка вариативности для дистанций
    if len(np.unique(control_distances)) < 2 or len(np.unique(patient_distances)) < 2:
        return {
            'Переменная': column_name,
            'Тип анализа': 'Векторный',
            'Контрольная группа (M±SD)': 'Нет вариации',
            'Группа пациентов (M±SD)': 'Нет вариации',
            'Тест на нормальность (p)': '-',
            'Стат. тест и p-value': '-',
            'Значимость': 'Нет'
        }

    stat, p_value = mannwhitneyu(control_distances, patient_distances, alternative='two-sided')

    return {
        'Переменная': column_name,
        'Тип анализа': 'Векторный',
        'Контрольная группа (M±SD)': f"{control_distances.mean():.3f} ± {control_distances.std():.3f}",
        'Группа пациентов (M±SD)': f"{patient_distances.mean():.3f} ± {patient_distances.std():.3f}",
        'Тест на нормальность (p)': '-',
        'Стат. тест и p-value': f"Mann–Whitney, p={p_value:.3f}",
        'Значимость': 'Да' if p_value < 0.05 else 'Нет'
    }

for feature in vector_features:
    results.append(analyze_cosine_distance(feature))

# === Вывод итоговой таблицы ===
results_df = pd.DataFrame(results)
# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)
# results_df

results_df.to_excel('results_analysis.xlsx', index=False)


In [18]:
significant_vars = results_df[results_df['Значимость'] == 'Да']['Переменная'].tolist()
with open('significant_vars.txt', 'w', encoding='utf-8') as f:
    for var in significant_vars:
        f.write(f"{var}\n")